# Advent of code 2022

I usually avoid using `eval`, but sometimes needs must. Otherwise a pretty standard recursion task.

In [1]:
import itertools as it

## Part 1

In [2]:
test_input='''[1,1,3,1,1]
[1,1,5,1,1]

[[1],[2,3,4]]
[[1],4]

[9]
[[8,7,6]]

[[4,4],4,4]
[[4,4],4,4,4]

[7,7,7,7]
[7,7,7]

[]
[3]

[[[]]]
[[]]

[1,[2,[3,[4,[5,6,7]]]],8,9]
[1,[2,[3,[4,[5,6,0]]]],8,9]'''

In [3]:
with open('data/day13.txt') as fIn:
    puzzle_input=fIn.read()

In [5]:
def parse_input(str_in):
    
    return [(eval(nl.split()[0]), eval(nl.split()[1]))
            for nl in str_in.strip().split('\n\n')]

In [6]:
parse_input(test_input)

[([1, 1, 3, 1, 1], [1, 1, 5, 1, 1]),
 ([[1], [2, 3, 4]], [[1], 4]),
 ([9], [[8, 7, 6]]),
 ([[4, 4], 4, 4], [[4, 4], 4, 4, 4]),
 ([7, 7, 7, 7], [7, 7, 7]),
 ([], [3]),
 ([[[]]], [[]]),
 ([1, [2, [3, [4, [5, 6, 7]]]], 8, 9], [1, [2, [3, [4, [5, 6, 0]]]], 8, 9])]

Define the main comparison function as a predicate. The signal is always a list, and I'm going to assume that there aren't any identical pairs.

In [7]:
def in_right_order(left, right):
    
    # This would be so much more elegant in prolog...
    
    if left==right:
        return None
    
    elif isinstance(left, int) and isinstance(right, list):
        return in_right_order([left], right)
    
    elif isinstance(left, list) and isinstance(right, int):
        return in_right_order(left, [right])
    
    elif isinstance(left, int) and isinstance(right, int):
        if left < right:
            return 'left'
        else:
            return 'right'
    
    elif isinstance(left, list) and isinstance(right, list):
        
        if left==[]:
            return 'left'
        elif right==[]:
            return 'right'
        else:
            iro=in_right_order(left.pop(0), right.pop(0))
            if iro:
                return iro
            else:
                return in_right_order(left, right)        

In [8]:
def day13_a(str_in):
    
    iros=[in_right_order(l, r)=='left' for (l, r) in parse_input(str_in)]
    
    return sum([idx for (iro, idx) in zip(iros, it.count(1)) if iro]) 



In [9]:
assert day13_a(test_input)==13

In [10]:
day13_a(puzzle_input)

5806

## Part 2

Just setting the right sorting key, surely..?

There's a nice function in `functools` which maps a comparator onto a key function

In [11]:
import functools as ft
import copy

In [12]:
sorted([x for y in parse_input(test_input) for x in y],
       key=ft.cmp_to_key(lambda x, y:-1 if in_right_order(x, y) else 1))

[[[2, [3, [4, [5, 6, 0]]]], 8, 9],
 [8, 9],
 [],
 [],
 [],
 [],
 [7],
 [],
 [],
 [],
 [],
 [4],
 [],
 [],
 [],
 [1, 1]]

Ooops! The comparison function consumes the inputs, so let's do a quick tweak:

In [13]:
def in_right_order_b(left, right):
    
    if in_right_order(copy.deepcopy(left),
                      copy.deepcopy(right))=='left':
        return -1
    else:
        return 1

In [14]:
sorted([x for y in parse_input(test_input) for x in y],
       key=ft.cmp_to_key(in_right_order_b))

[[],
 [[]],
 [[[]]],
 [1, 1, 3, 1, 1],
 [1, 1, 5, 1, 1],
 [[1], [2, 3, 4]],
 [1, [2, [3, [4, [5, 6, 0]]]], 8, 9],
 [1, [2, [3, [4, [5, 6, 7]]]], 8, 9],
 [[1], 4],
 [3],
 [[4, 4], 4, 4],
 [[4, 4], 4, 4, 4],
 [7, 7, 7],
 [7, 7, 7, 7],
 [[8, 7, 6]],
 [9]]

That's better. Now just put in the divider packets and find the final indices:

In [15]:
def day13_b(str_in):
    p=[x for y in parse_input(str_in) for x in y]
    p+=[ [[2]], [[6]] ]
    
    p.sort(key=ft.cmp_to_key(in_right_order_b))
    
    return (p.index([[2]])+1) * (p.index([[6]])+1)

In [16]:
assert day13_b(test_input)==140

In [17]:
day13_b(puzzle_input)

23600